In [1]:
import pandas as pd
import argparse
import os

path_faire = '~/sedtrap/metadata/FAIRe-NOAA_noaa-aoml-ngmt_20260109.xlsx'
sample_name_column = 'lib_id'
absolute_path_sequences = '/data/sequences/MichiganState'
output_directory = '.'
column_suffix = 'pcr_plate_id'
direction_suffix = 'right'
num_chars_suffix = 8
delimiter_suffix = '_PCR'
         
df_project = pd.read_excel(path_faire, sheet_name='projectMetadata', comment='#')
df_sample = pd.read_excel(path_faire, sheet_name='sampleMetadata', comment='#')
df_exptrun = pd.read_excel(path_faire, sheet_name='experimentRunMetadata', comment='#')

df_project_first_2_cols_removed = df_project.iloc[:, 2:]
df_project_first_2_cols_removed.set_index('term_name', inplace=True)
df_project_wide = df_project_first_2_cols_removed.transpose()
project_id = df_project_wide['project_id'].iloc[0]
first_row_values = df_project_wide.loc['project_level']
df_project_filled = df_project_wide.fillna(first_row_values)

seq_run_ids = df_exptrun['seq_run_id'].unique()
assay_names = df_exptrun['assay_name'].unique()
if set(assay_names) != set(df_exptrun['assay_name'].dropna()):
    raise ValueError("Inconsistent assay names between projectMetadata and experimentRunMetadata.")

# Metadata file for each assay_name
dict_assay_short = {}
for assay_name in df_exptrun['assay_name'].unique():
    # For each assay, merge experiment run metadata with sample metadata and project metadata
    metadata = df_exptrun[df_exptrun['assay_name'] == assay_name].merge(df_sample, on='samp_name', how='left', suffixes=('', '_SAMPLE'))
    metadata = pd.merge(metadata, df_project_filled.loc[assay_name].to_frame().transpose(), how='cross', suffixes=('', '_PROJECT'))
    metadata.dropna(axis=1, how='all', inplace=True)
    
    # Check if sample_name_column has duplicates and exit if so
    if metadata[sample_name_column].duplicated().any():
        print(f"Warning: Duplicated entries found in column '{sample_name_column}' for assay '{assay_name}'.")

    # Insert sample_name column based on sample_name_column variable
    metadata.insert(0, 'sample_name', metadata[sample_name_column])

    # Add suffix to samp_names that are duplicated within the assay using column_suffix (if provided), direction_suffix, num_chars_suffix, and delimiter_suffix
    if column_suffix is not None:
        metadata.insert(2, 'samp_name_unique', metadata['samp_name'])        
        duplicated_samples = metadata['samp_name'][metadata['samp_name'].duplicated(keep=False)].unique()
        for sample in duplicated_samples:
            sample_rows = metadata[metadata['samp_name'] == sample]
            for idx, row in sample_rows.iterrows():
                if direction_suffix == 'right':
                    suffix = f'{delimiter_suffix}{row[column_suffix][-num_chars_suffix:]}'  # Placeholder suffix
                else:               
                    suffix = f'{delimiter_suffix}{row[column_suffix][:num_chars_suffix]}'  # Placeholder suffix
                new_samp_name = f"{row['samp_name']}{suffix}"
                metadata.at[idx, 'samp_name_unique'] = new_samp_name
    
    # Short assay name to use for output files
    gene = df_project_filled.loc[assay_name]['target_gene'].split(' ')[0]
    subfragment = df_project_filled.loc[assay_name]['target_subfragment']
    subfragment_part = subfragment.split(' ')[0].replace('-', '').replace('_', '') if isinstance(subfragment, str) and subfragment.strip() else ''
    if subfragment_part:
        dict_assay_short[assay_name] = f"{gene}-{subfragment_part}"
    else:
        dict_assay_short[assay_name] = gene
    metadata.to_csv(os.path.join(output_directory, f"{project_id}_{dict_assay_short[assay_name]}_metadata.tsv"), sep='\t', index=False)
    print(f"Generated metadata file {project_id}_{dict_assay_short[assay_name]}_metadata.tsv")

# Manifest file for each seq_run_id
for seq_run_id in seq_run_ids:
    for assay_name in df_exptrun[df_exptrun['seq_run_id'] == seq_run_id]['assay_name'].unique():
        manifest = df_exptrun[(df_exptrun['seq_run_id'] == seq_run_id) & (df_exptrun['assay_name'] == assay_name)][['samp_name', 'filename', 'filename2']]
        manifest['filename'] = manifest['filename'].apply(lambda x: os.path.join(absolute_path_sequences, f'{seq_run_id}', str(x)))
        manifest['filename2'] = manifest['filename2'].apply(lambda x: os.path.join(absolute_path_sequences, f'{seq_run_id}', str(x)))
        manifest.columns = ['sample-id', 'forward-absolute-filepath', 'reverse-absolute-filepath']
        manifest.to_csv(os.path.join(output_directory, f"{seq_run_id}_{dict_assay_short[assay_name]}_manifest.tsv"), sep='\t', index=False)
        print(f"Generated manifest file {seq_run_id}_{dict_assay_short[assay_name]}_manifest.tsv")

Generated metadata file noaa-aoml-ngmt_16S-V4V5_metadata.tsv
Generated metadata file noaa-aoml-ngmt_18S-V9_metadata.tsv
Generated manifest file 20220912_16S-Amplicon_PE250_16S-V4V5_manifest.tsv
Generated manifest file 20231110_AND14513_Amplicon_PE250_16S-V4V5_manifest.tsv
Generated manifest file 20240325_16S-Amplicon_PE250_16S-V4V5_manifest.tsv
Generated manifest file 20250423_16S-Amplicon_PE250_16S-V4V5_manifest.tsv
Generated manifest file 20250728_THO16883_16S-Amplicon_PE250_16S-V4V5_manifest.tsv
Generated manifest file 20220916_18S-Amplicon_PE250_18S-V9_manifest.tsv
Generated manifest file 20231107_AND14514_Amplicon_PE150_18S-V9_manifest.tsv
Generated manifest file 20240401_18S-Amplicon_PE150_18S-V9_manifest.tsv
Generated manifest file 20250423_18S-Amplicon_PE150_18S-V9_manifest.tsv
Generated manifest file 20250730_THO16884_18S-Amplicon_PE150_18S-V9_manifest.tsv


In [2]:
metadata[sample_name_column][metadata[sample_name_column].duplicated()]

Series([], Name: lib_id, dtype: object)

In [3]:
metadata[metadata['sample_name'].str.contains('GMT_CTD_18S_Blk')]

,sample_name,samp_name,samp_name_unique,assay_name,pcr_plate_id,pcr_well_position,pcr_well_number,lib_id,seq_run_id,filename,...,sequencing_location,platform,instrument,seq_kit,adapter_forward,adapter_reverse,lib_screen,checksum_method,seq_method_additional,ship_crs_expocode
196,GMT_CTD_18S_Blk1_S57,NGMT_GMT234_CTD_ExtractBlank_A,NGMT_GMT234_CTD_ExtractBlank_A,Eukarya-18S-V9-Lane-Medlin,"GMT2,3,4 CTD 18S 20230803",D7,52,GMT_CTD_18S_Blk1_S57,20240401_18S-Amplicon_PE150,GMT_CTD_18S_Blk1_S57_L001_R1_001.fastq.gz,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,32PE | 320G
197,GMT_CTD_18S_Blk2_S58,NGMT_GMT234_CTD_ExtractBlank_B,NGMT_GMT234_CTD_ExtractBlank_B,Eukarya-18S-V9-Lane-Medlin,"GMT2,3,4 CTD 18S 20230803",E7,53,GMT_CTD_18S_Blk2_S58,20240401_18S-Amplicon_PE150,GMT_CTD_18S_Blk2_S58_L001_R1_001.fastq.gz,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,32PE | 320G


In [4]:
metadata

,sample_name,samp_name,samp_name_unique,assay_name,pcr_plate_id,pcr_well_position,pcr_well_number,lib_id,seq_run_id,filename,...,sequencing_location,platform,instrument,seq_kit,adapter_forward,adapter_reverse,lib_screen,checksum_method,seq_method_additional,ship_crs_expocode
0,GMT1_18S_1A_S1,NGMT_GMT1_Sed_550m_01A,NGMT_GMT1_Sed_550m_01A,Eukarya-18S-V9-Lane-Medlin,GMT1 Sed 18S 20220808,A1,1,GMT1_18S_1A_S1,20220916_18S-Amplicon_PE250,GMT1_18S_1A_S1_L001_R1_001.fastq.gz,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,32PE | 320G
1,GMT1_18S_1B_S2,NGMT_GMT1_Sed_550m_01B,NGMT_GMT1_Sed_550m_01B,Eukarya-18S-V9-Lane-Medlin,GMT1 Sed 18S 20220808,B1,2,GMT1_18S_1B_S2,20220916_18S-Amplicon_PE250,GMT1_18S_1B_S2_L001_R1_001.fastq.gz,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,32PE | 320G
2,GMT1_18S_1C_S3,NGMT_GMT1_Sed_550m_01C,NGMT_GMT1_Sed_550m_01C,Eukarya-18S-V9-Lane-Medlin,GMT1 Sed 18S 20220808,C1,3,GMT1_18S_1C_S3,20220916_18S-Amplicon_PE250,GMT1_18S_1C_S3_L001_R1_001.fastq.gz,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,32PE | 320G
3,GMT1_18S_2A_S4,NGMT_GMT1_Sed_550m_02A,NGMT_GMT1_Sed_550m_02A,Eukarya-18S-V9-Lane-Medlin,GMT1 Sed 18S 20220808,D1,4,GMT1_18S_2A_S4,20220916_18S-Amplicon_PE250,GMT1_18S_2A_S4_L001_R1_001.fastq.gz,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,32PE | 320G
4,GMT1_18S_2B_S5,NGMT_GMT1_Sed_550m_02B,NGMT_GMT1_Sed_550m_02B,Eukarya-18S-V9-Lane-Medlin,GMT1 Sed 18S 20220808,E1,5,GMT1_18S_2B_S5,20220916_18S-Amplicon_PE250,GMT1_18S_2B_S5_L001_R1_001.fastq.gz,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,32PE | 320G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,GMT6_17C_18S_S51,NGMT_GMT6_Sed_550m_17C,NGMT_GMT6_Sed_550m_17C,Eukarya-18S-V9-Lane-Medlin,GMT6 Sed + GMT7 CTD 18S 20250626,C7,51,GMT6_17C_18S_S51,20250730_THO16884_18S-Amplicon_PE150,GMT6_17C_18S_S51_L001_R1_001.fastq.gz,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,32PE | 320G
394,GMT6_Blk1_18S_S2,NGMT_GMT6_Sed_550m_ExtractBlank_1,NGMT_GMT6_Sed_550m_ExtractBlank_1,Eukarya-18S-V9-Lane-Medlin,GMT6 Sed + GMT7 CTD 18S 20250626,B1,2,GMT6_Blk1_18S_S2,20250730_THO16884_18S-Amplicon_PE150,GMT6_Blk1_18S_S2_L001_R1_001.fastq.gz,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,32PE | 320G
395,GMT6_Blk2_18S_S3,NGMT_GMT6_Sed_550m_ExtractBlank_2,NGMT_GMT6_Sed_550m_ExtractBlank_2,Eukarya-18S-V9-Lane-Medlin,GMT6 Sed + GMT7 CTD 18S 20250626,C1,3,GMT6_Blk2_18S_S3,20250730_THO16884_18S-Amplicon_PE150,GMT6_Blk2_18S_S3_L001_R1_001.fastq.gz,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,ACACTGACGACATG